In [1]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@Yunus111",
                               database = "project_2")

cursor = conn.cursor()

In [2]:
df_sales = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\dataset\Sales.csv')
df_sales.columns = df_sales.columns.str.lower()
df_sales.columns = df_sales.columns.str.replace(" ","_")
df_sales.columns
df_sales['order_date']= pd.to_datetime(df_sales['order_date'])
df_sales['delivery_date']= pd.to_datetime(df_sales['delivery_date'])

In [3]:
df_sales['order_date']= pd.to_datetime(df_sales['order_date'])
df_sales['delivery_date']= pd.to_datetime(df_sales['delivery_date'])
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [4]:
df_sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD,NaN
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12.0
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12.0
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11.0
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11.0


In [5]:
#Ignore Duplicate Entries

import mysql.connector
from mysql.connector import Error
import pandas as pd

# SQL query to create the sales table if it doesn't exist
create_table_query = '''
CREATE TABLE IF NOT EXISTS sales (
    order_number INT,
    line_item INT,
    order_date DATE,
    delivery_date DATE,
    customerkey INT,
    storekey INT,
    productkey INT,
    quantity INT,
    currency_code VARCHAR(10),
    shipping_days INT,
    PRIMARY KEY (order_number, line_item)
)
'''

# SQL query for inserting data into the sales table
insert_sales_values = '''
INSERT IGNORE INTO sales (
    order_number, 
    line_item, 
    order_date, 
    delivery_date, 
    customerkey, 
    storekey, 
    productkey, 
    quantity, 
    currency_code, 
    shipping_days
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

try:
    # Establish the connection
    conn = mysql.connector.connect(
        host='127.0.0.1',
        database='project_2',
        user='root',
        password='Admin@Yunus111'
    )
    
    if conn.is_connected():
        # Create a cursor object
        cursor = conn.cursor()
        
        # Create the table if it doesn't exist
        cursor.execute(create_table_query)

        # Replace NaN values with None (or appropriate default values)
        df_sales = df_sales.fillna({
            'order_number': 0,
            'line_item': 0,
            'order_date': pd.Timestamp('1970-01-01'),
            'delivery_date': pd.Timestamp('1970-01-01'),
            'customerkey': 0,
            'storekey': 0,
            'productkey': 0,
            'quantity': 0,
            'currency_code': 'USD',
            'shipping_days': 0
        })

        # Insert data into the sales table
        for index, row in df_sales.iterrows():
            cursor.execute(insert_sales_values, (
                row['order_number'],
                row['line_item'],
                row['order_date'],
                row['delivery_date'],
                row['customerkey'],
                row['storekey'],
                row['productkey'],
                row['quantity'],
                row['currency_code'],
                row['shipping_days']
            ))

        # Commit the transaction
        conn.commit()

except mysql.connector.Error as error:
    print(f"Error: {error}")

finally:
    # Ensure cursor and connection are closed properly
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()